In [1]:
# -*- coding: utf-8 -*- 
from lib import geoSearch
import json
import logging

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
geoSearch.logger.setLevel(logging.DEBUG)
from shapely.geometry import mapping, shape

In [3]:
#GS.test_kit('test_complete.csv')

In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [9]:
text = u"cada mes la im arroja al bolsillo de sus funcionarios 50 veces el importe arrojado a la plaza zabala para instalar el adefesio adefesio usd420 mil retribuciones mensuales usd20 millones en ambos casos dinero sustraido al contribuyente para devolverle una ciudad basural"
analyzedText(text)

u'cada mes_la la_im arroja_al al_bolsillo bolsillo funcionarios veces_el el_importe importe arrojado_a a_la la_plaza plaza_zabala zabala instalar_el el_adefesio adefesio adefesio usd420 mil retribuciones mensuales usd20 millones ambos casos dinero sustraido_al al_contribuyente contribuyente devolverle una_ciudad ciudad_basural'

In [10]:
matches = GS.complete_search(text,50)
print(GS.strip_name(text, ' '))
for key, match in matches:
    print('✔️ MATCH: {:.3f}{: >20}{: >50}'.format(
        match['score'], 
        match['field'], 
        match['name']
    ))
print('\n')

DEBUG:GeoSearch:cine plaza & plaza cagancha 1129 ## 6.788245 
 -->                 text
              normal      plaza => 0.500
-->         text_aliases
              normal      plaza => 0.263

DEBUG:GeoSearch:FIELD:text
min_idx = 88
max_idx = 88
len_longest = 5
highlights_count = 1
importance = 0.5
len_all = 5
DEBUG:GeoSearch:FIELD:text_aliases
min_idx = 88
max_idx = 88
len_longest = 5
highlights_count = 1
importance = 0.263157894737
len_all = 5
DEBUG:GeoSearch:TOTAL = 212.13265625
First Score = 84.8530625
Second Score = 44.6595065789
First Completeness = 0.5
Second Completeness = 0.263157894737
Boost Field = 10
DEBUG:GeoSearch:teatro ateneo & plaza cagancha 1157 ## 4.08015 
 -->         text_aliases
              normal      plaza => 0.263

DEBUG:GeoSearch:FIELD:text_aliases
min_idx = 88
max_idx = 88
len_longest = 5
highlights_count = 1
importance = 0.263157894737
len_all = 5
DEBUG:GeoSearch:TOTAL = 10.8712229222
First Score = 0
Second Score = 26.8430921053
First Completeness = 0.0

DEBUG:GeoSearch:radisson montevideo victoria plaza hotel & radisson montevideo victoria plaza hotel ## 4.2059526 
 -->                 text
              normal      plaza => 0.125
-->         text_aliases
              normal      plaza => 0.125

DEBUG:GeoSearch:FIELD:text
min_idx = 88
max_idx = 88
len_longest = 5
highlights_count = 1
importance = 0.125
len_all = 5
DEBUG:GeoSearch:FIELD:text_aliases
min_idx = 88
max_idx = 88
len_longest = 5
highlights_count = 1
importance = 0.125
len_all = 5
DEBUG:GeoSearch:TOTAL = 2.05368779297
First Score = 13.143601875
Second Score = 13.143601875
First Completeness = 0.125
Second Completeness = 0.125
Boost Field = 10


cada mes la im arroja al bolsillo de sus funcionarios 50 veces el importe arrojado a la plaza zabala para instalar el adefesio adefesio usd420 mil retribuciones mensuales usd20 millones en ambos casos dinero sustraido al contribuyente para devolverle una ciudad basural
✔️ MATCH: 212.133     lugares_interes                    cine-plaza&plaza-cagancha-1129
✔️ MATCH: 122.762            geonames                         plaza-matriz&plaza-matriz
✔️ MATCH: 122.762            geonames                         plaza-fuerte&plaza-fuerte
✔️ MATCH: 77.308            geonames                     plaza-libertad&plaza-libertad
✔️ MATCH: 42.992            geonames                 palm-beach-plaza&palm-beach-plaza
✔️ MATCH: 36.374            geonames             plaza-de-la-armada&plaza-de-la-armada
✔️ MATCH: 30.928            geonames           plaza-del-entrevero&plaza-del-entrevero
✔️ MATCH: 30.928            geonames           plaza-independencia&plaza-independencia
✔️ MATCH: 25.673            geo

In [ ]:
for res in GS.search_adversary_fields(text, 'cruces_vias',500):
    obj = GS.get_result_object(res)
    print(GS.print_object(obj))

In [ ]:
print(matches)